# Deploy credit risk classification model in Azure Container Instance (ACI)

Now, you're ready to deploy the model as a web service in [Azure Container Instances](https://docs.microsoft.com/azure/container-instances/) (ACI). A web service is an image, in this case a Docker image, that encapsulates the scoring logic and the model itself. 

In this part of the tutorial, you use Azure Machine Learning service to:

> * Set up your testing environment
> * Retrieve the model from your workspace
> * Test the model locally
> * Deploy the model to ACI
> * Test the deployed model

ACI is a great solution for testing and understanding the workflow. For scalable production deployments, consider using Azure Kubernetes Service. For more information, see [how to deploy and where](https://docs.microsoft.com/azure/machine-learning/service/how-to-deploy-and-where).

## Upgrade scikit-learn

In [1]:
!pip install --upgrade scikit-learn==0.20.2

Requirement already up-to-date: scikit-learn==0.20.2 in /home/nbuser/anaconda3_501/lib/python3.6/site-packages (0.20.2)
You should consider upgrading via the 'pip install --upgrade pip' command.


## Action: Restart the kernel

## Get scikit-learn 0.20 credit risk model 

In [2]:
!rm -rf german_credit_risk.joblib
!wget https://github.com/pmservice/ai-openscale-tutorials/raw/master/applications/custom-ml-engine-bluemix/models/credit/german_credit_risk.joblib

--2019-05-16 08:27:10--  https://github.com/pmservice/ai-openscale-tutorials/raw/master/applications/custom-ml-engine-bluemix/models/credit/german_credit_risk.joblib
Resolving webproxy (webproxy)... 10.36.68.1
Connecting to webproxy (webproxy)|10.36.68.1|:3128... connected.
Proxy request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/applications/custom-ml-engine-bluemix/models/credit/german_credit_risk.joblib [following]
--2019-05-16 08:27:11--  https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/applications/custom-ml-engine-bluemix/models/credit/german_credit_risk.joblib
Connecting to webproxy (webproxy)|10.36.68.1|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 9183 (9.0K) [application/octet-stream]
Saving to: ‘german_credit_risk.joblib’

german_credit_risk. 100%[===================>]   8.97K  --.-KB/s    in 0.02s   

2019-05-16 08:27:11 (417 KB/s) - ‘german_c

## Register model

In [3]:
model_name = "german_credit_risk"
model_path="german_credit_risk.joblib"

In [4]:
import os
import urllib.request

# register a model
from azureml.core import Workspace
ws = Workspace.from_config()

from azureml.core.model import Model

model = Model.register(model_path=model_path,
                        model_name=model_name,
                        tags={"data": "german_credit_risk", "model": "classification"},
                        description="credit risk sample scikit model",
                        workspace=ws)

Registering model german_credit_risk


In [5]:
ws.get_details()

{'id': '/subscriptions/744bca72-2299-451c-b682-ed6fb75fb671/resourceGroups/ai-ops-squad/providers/Microsoft.MachineLearningServices/workspaces/ai-ops-squad-server',
 'name': 'ai-ops-squad-server',
 'location': 'southcentralus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'workspaceid': '16b06534-668c-4887-aa3c-a10d99eb27e4',
 'description': '',
 'friendlyName': '',
 'creationTime': '2018-10-18T12:12:25.7036291+00:00',
 'containerRegistry': '/subscriptions/744bca72-2299-451c-b682-ed6fb75fb671/resourcegroups/ai-ops-squad/providers/microsoft.containerregistry/registries/aiopssquadserv4124490709',
 'keyVault': '/subscriptions/744bca72-2299-451c-b682-ed6fb75fb671/resourcegroups/ai-ops-squad/providers/microsoft.keyvault/vaults/aiopssquadserv3633962467',
 'applicationInsights': '/subscriptions/744bca72-2299-451c-b682-ed6fb75fb671/resourcegroups/ai-ops-squad/providers/microsoft.insights/components/aiopssquadserv0294391293',
 'identityPrincipalId': 'e057536e-7de0-4980-8a09-eabf175

## Set up the environment

Start by setting up a testing environment.

### Import packages

Import the Python packages needed for this tutorial.

In [6]:
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
import os 

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.33


### Retrieve the model

You registered a model in your workspace in the previous tutorial. Now, load this workspace and download the model to your local directory.

In [7]:
ws = Workspace.from_config()
model=Model(ws, model_name)

model.download(target_dir=os.getcwd(), exist_ok=True)

# verify the downloaded model file
file_path = os.path.join(os.getcwd(), "german_credit_risk.joblib")
os.stat(file_path)

os.stat_result(st_mode=33188, st_ino=13, st_dev=49, st_nlink=1, st_uid=1200, st_gid=1200, st_size=9183, st_atime=0, st_mtime=1557995237, st_ctime=1557995237)

## Test model locally

Before deploying, make sure your model is working locally by:
* Loading test data
* Predicting test data

### Load test data

Load the test data from the git repository.

In [8]:
!rm -rf credit_risk_training.csv
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/notebooks/data/credit_risk_training.csv

--2019-05-16 08:27:17--  https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/notebooks/data/credit_risk_training.csv
Resolving webproxy (webproxy)... 10.36.68.1
Connecting to webproxy (webproxy)|10.36.68.1|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 694222 (678K) [text/plain]
Saving to: ‘credit_risk_training.csv’

credit_risk_trainin 100%[===================>] 677.95K  2.65MB/s    in 0.3s    

2019-05-16 08:27:18 (2.65 MB/s) - ‘credit_risk_training.csv’ saved [694222/694222]



In [9]:
import pandas as pd

data_df = pd.read_csv("credit_risk_training.csv",
                    dtype={'LoanDuration': int, 'LoanAmount': int, 'InstallmentPercent': int, 'CurrentResidenceDuration': int, 'Age': int, 'ExistingCreditsCount': int, 'Dependents': int})

### Predict test data

Feed the test dataset to the model to get predictions.

In [10]:
import pickle
from sklearn.externals import joblib

clf = joblib.load( os.path.join(os.getcwd(), model_path))

In [11]:
test_data = data_df
test_data = test_data.drop('Risk', axis=1)

In [12]:
import numpy as np
from sklearn.metrics import accuracy_score

scores = clf['model'].predict_proba(test_data).tolist()
predictions = clf['postprocessing'](clf['model'].predict(test_data))

accuracy_score(data_df.Risk, predictions)

0.7784

## Deploy as web service

Once you've tested the model, deploy the model as a web service hosted in ACI. 

To build the correct environment for ACI, provide the following:
* A scoring script to show how to use the model
* An environment file to show what packages need to be installed
* A configuration file to build the ACI
* The model you trained before

### Create scoring script

Create the scoring script, called score.py, used by the web service call to show how to use the model.

You must include two required functions into the scoring script:
* The `init()` function, which typically loads the model into a global object. This function is run only once when the Docker container is started. 

* The `run(input_data)` function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats are supported.


### Azure studio-like format

In [13]:
%%writefile score_azure.py
import json
import numpy as np
import os
import pandas as pd
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('german_credit_risk')
    model = joblib.load(model_path)

def run(input_data):
    try:
        if type(input_data) is str:
            dict_data = json.loads(input_data)
        else:
            dict_data = input_data
            
        data = pd.DataFrame.from_dict(dict_data['input'])   
        predictions = model['postprocessing'](model['model'].predict(data))
        scores = model['model'].predict_proba(data).tolist()
        records = []
        
        for pred, prob in zip(predictions, scores):
            records.append({"Scored Labels": pred, "Scored Probabilities": prob})
 
        result = {'output': records}
        
        return json.dumps(result)
    except Exception as e:
        result = str(e)
        # return error message back to the client
        return json.dumps({"error": result})

Overwriting score_azure.py


### Create environment file

Next, create an environment file, called myenv.yml, that specifies all of the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. This model needs `scikit-learn` and `azureml-sdk`.

In [16]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn==0.20.2")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

Review the content of the `myenv.yml` file.

In [17]:
with open("myenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults
- scikit-learn==0.20.2



### Create configuration file

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. While it depends on your model, the default of 1 core and 1 gigabyte of RAM is usually sufficient for many models. If you feel you need more later, you would have to recreate the image and redeploy the service.

In [18]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "german credit risk",  "method" : "sklearn"}, 
                                               description='Predict Credit Risk with sklearn on Azure Service')

### Deploy in ACI
Estimated time to complete: **about 7-8 minutes**

Configure the image and deploy. The following code goes through these steps:

1. Build an image using:
   * The scoring file (`score.py`)
   * The environment file (`myenv.yml`)
   * The model file
1. Register that image under the workspace. 
1. Send the image to the ACI container.
1. Start up a container in ACI using the image.
1. Get the web service HTTP endpoint.

## Scoring endpoint creation

In [19]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage


deployment_name = 'credit-risk-prediction'

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score_azure.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

service_az = Webservice.deploy_from_model(workspace=ws,
                                       name=deployment_name,
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service_az.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image credit-risk-prediction:1, operation "Succeeded"
Creating service
Running......................
SucceededACI service creation operation finished, operation "Succeeded"
CPU times: user 1.9 s, sys: 650 ms, total: 2.56 s
Wall time: 6min 55s


Get the scoring web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [20]:
print(service_az.scoring_uri)

http://20.189.138.213:80/score


## Test deployed service

Earlier you scored all the test data with the local version of the model. Now, you can test the deployed model with a sample from the test data.  

The following code goes through these steps:
1. Send the data as a JSON array to the web service hosted in ACI. 

1. Print the returned predictions and plot them along with the input images. Red font and inverse image (white on black) is used to highlight the misclassified samples. 



In [21]:
scorig_data = {"input":[{
                            'CheckingStatus': "0_to_200", 'LoanDuration': 31, 'CreditHistory': "credits_paid_to_date", 'LoanPurpose': "other",
                            'LoanAmount': 1889, 'ExistingSavings': "100_to_500",'EmploymentDuration': "less_1",'InstallmentPercent': 3,'Sex': "female",
                            'OthersOnLoan': "none",'CurrentResidenceDuration': 3, 'OwnsProperty': "savings_insurance", 'Age': 32,'InstallmentPlans': "none",
                            'Housing': "own",'ExistingCreditsCount': 1,'Job': "skilled",'Dependents': 1,'Telephone': "none",'ForeignWorker': "yes",
                        },
                        {
                            'CheckingStatus': "no_checking", 'LoanDuration': 13, 'CreditHistory': "credits_paid_to_date", 'LoanPurpose': "car_new",
                            'LoanAmount': 1389, 'ExistingSavings': "100_to_500",'EmploymentDuration': "1_to_4",'InstallmentPercent': 2,'Sex': "male",
                            'OthersOnLoan': "none",'CurrentResidenceDuration': 3, 'OwnsProperty': "savings_insurance", 'Age': 25,'InstallmentPlans': "none",
                            'Housing': "own",'ExistingCreditsCount': 2,'Job': "skilled",'Dependents': 2,'Telephone': "none",'ForeignWorker': "yes",
                        }]
              }

In [22]:
import requests

headers = {'Content-Type':'application/json'}

# for AKS deployment you'd need to the service key in the header as well
# api_key = service.get_key()
# headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 

resp = requests.post(service_az.scoring_uri, json=scorig_data, headers=headers)

print("POST to url", service_az.scoring_uri)
print(resp.json())

POST to url http://20.189.138.213:80/score
{"output": [{"Scored Labels": "No Risk", "Scored Probabilities": [0.8922524675865824, 0.10774753241341757]}, {"Scored Labels": "No Risk", "Scored Probabilities": [0.8335192848546905, 0.1664807151453095]}]}
